In [1]:
from hsl_leap.leap_hand import LeapHand, LeapHandConfig, MJ_ZERO_POSITION
import time
import numpy as np

/Users/ianc/Documents/GitHub/hsl_leap/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SAVE_DIR = "outputs/if_mcp"

# -------------------------------------------------
# Start configuration (ALL joints)
# -------------------------------------------------
start_cfg = {
    "if_mcp": 0.0,
    "if_rot": 0.0,
    "if_pip": 0.0,
    "if_dip": 0.0,
    "mf_mcp": 0.0,
    "mf_rot": 0.0,
    "mf_pip": 0.0,
    "mf_dip": 0.0,
    "rf_mcp": 0.0,
    "rf_rot": 0.0,
    "rf_pip": 0.0,
    "rf_dip": 0.0,
    "th_cmc": 0.0,
    "th_axl": 0.0,
    "th_mcp": 0.13,
    "th_ipl": 0.0,
}

# -------------------------------------------------
# Control configuration (ONLY controlled joints)
# -------------------------------------------------
ctrl_cfg = {
    "if_mcp": {"low": 0.0, "high": 1.0},
    # "if_pip": {"low": 0.0, "high": 0.8},
    # "if_dip": {"low": 0.0, "high": 0.8},
}

In [3]:
hand = LeapHand(
        LeapHandConfig(
            port="/dev/tty.usbserial-FTAO51BR",
            use_mj_motor_config=True,
        )
    )
hand.connect()

actions = {f"{k}.pos": np.rad2deg(v) for k, v in start_cfg.items()}
hand.move(actions, duration=2.0)

Read observation in 0.0159 seconds


In [4]:
# -------------------------------------------------
# Create control signal
# -------------------------------------------------
# -------------------------------------------------
# Time
# -------------------------------------------------
duration = 5.0
timestep = 0.002
n_steps = int(duration / timestep)
t = np.arange(n_steps) * timestep
ctrl = np.zeros((n_steps, 16))
freq = 0.5  # Hz

ctrl = {}
for joint_name, cfg in ctrl_cfg.items():
    start = start_cfg[joint_name]
    low = cfg["low"]
    high = cfg["high"]

    # Sine wave between low and high
    amplitude = 0.5 * (high - low)
    offset = 0.5 * (high + low)

    # Phase so it starts at initial joint position
    phase = np.arcsin(np.clip((start - offset) / amplitude, -1.0, 1.0))

    ctrl[joint_name] = offset + amplitude * np.sin(
        2 * np.pi * freq * t + phase
    )

In [5]:
# go through steps at real-time speed
sensor = {k: [] for k in start_cfg.keys()}
sensor_time = []

start_time = time.time()
for i in range(n_steps):
    loop_time = time.time()

    actions = {f"{k}.pos": np.rad2deg(ctrl[k][i]) for k in ctrl_cfg.keys()}
    hand.send_action(actions)

    # read sensors  
    obs = hand.get_observation()
    for k in sensor.keys():
        sensor[k].append(np.deg2rad(obs[f"{k}.pos"]))
    sensor_time.append(time.time() - start_time)

    elapsed = time.time() - loop_time
    # wait for the rest of the timestep if we are ahead of schedule
    time.sleep(max(0.0, timestep - elapsed))

sensor = {k: np.array(v) for k, v in sensor.items()}
sensor_time = np.array(sensor_time)


Read observation in 0.0050 seconds
Read observation in 0.0157 seconds
Read observation in 0.0158 seconds
Read observation in 0.0159 seconds
Read observation in 0.0158 seconds
Read observation in 0.0159 seconds
Read observation in 0.0159 seconds
Read observation in 0.0159 seconds
Read observation in 0.0159 seconds
Read observation in 0.0159 seconds
Read observation in 0.0159 seconds
Read observation in 0.0160 seconds
Read observation in 0.0160 seconds
Read observation in 0.0159 seconds
Read observation in 0.0158 seconds
Read observation in 0.0159 seconds
Read observation in 0.0160 seconds
Read observation in 0.0160 seconds
Read observation in 0.0160 seconds
Read observation in 0.0160 seconds
Read observation in 0.0160 seconds
Read observation in 0.0161 seconds
Read observation in 0.0160 seconds
Read observation in 0.0159 seconds
Read observation in 0.0159 seconds
Read observation in 0.0160 seconds
Read observation in 0.0160 seconds
Read observation in 0.0159 seconds
Read observation in 

KeyboardInterrupt: 

In [4]:
hand.disconnect()